In [1]:
import time
import pandas as pd

In [2]:
ROOT = '/data1/wgs_APA/validation_res'

In [3]:
gem_1 = pd.read_csv(ROOT + '/gem0/protospacer_calls_per_cell.csv')
gem_1.head()

,cell_barcode,num_features,feature_call,num_umis
0,AAATGGAGTGGCGCTT-1,1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,1,Ascl2-2,27


In [4]:
from geomux import Geomux, read_table

gem_1['feature_call'] = gem_1['feature_call'].str.split('|')
gem_1['num_umis'] = gem_1['num_umis'].str.split('|')
gem_1 = gem_1.explode(list(['feature_call','num_umis']))
gem_1.drop(columns='num_features', inplace=True)
gem_1.rename(columns={"cell_barcode": "barcode", "feature_call": "guide", "num_umis": "n_umi"}, inplace=True)
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [5]:
# remove NA from matrix
gem_1 = gem_1[gem_1['guide'].notna()]
gem_1 = gem_1[gem_1['n_umi'].notna()]
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [6]:
# conver to cell by guide matrix, with n_umi as value, with NA as 0
gem_1_mat = gem_1.pivot(index='barcode', columns='guide', values='n_umi')
gem_1_mat.fillna(0, inplace=True)
gem_1_mat
# remove index and column name
gem_1_mat.index.name = None
gem_1_mat.columns.name = None
gem_1_mat

,Ascl2-2,Ascl2-3,Ascl2-4,BIRC2-2,BIRC2-4,BRIP1-2,BRIP1-4,CEP44-1,CEP44-2,CEP44-4,...,ZNF527-2,ZNF527-4,ZNF530-1,ZNF530-2,ZNF530-3,ZNF530-4,negative_control-1,negative_control-2,znf623-2,znf623-4
AAACCCAAGACGCTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGCTAATCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGACTGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGGAGGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGTGCGTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCAAGGACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCAGGAACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,42,0,0,0,0,0,0
TTTGTTGTCCAAAGGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCCCTCATG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,141,0,0,0,0,0,0


In [7]:
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [8]:
# set dtype of columns in gem_1, to dtype={"barcode": str, "guide": str, "n_umi": int},
gem_1 = gem_1.astype({"barcode": str, "guide": str, "n_umi": int})
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [9]:
matrix = gem_1.pivot_table(
    index="barcode", columns="guide", values="n_umi", fill_value=0
)
matrix

guide,Ascl2-2,Ascl2-3,Ascl2-4,BIRC2-2,BIRC2-4,BRIP1-2,BRIP1-4,CEP44-1,CEP44-2,CEP44-4,...,ZNF527-2,ZNF527-4,ZNF530-1,ZNF530-2,ZNF530-3,ZNF530-4,negative_control-1,negative_control-2,znf623-2,znf623-4
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGACGCTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGCTAATCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGACTGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGGAGGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGTGCGTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCAAGGACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCAGGAACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,42,0,0,0,0,0,0
TTTGTTGTCCAAAGGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
gx = Geomux(
    matrix,
    cell_names=matrix.index.values,
    guide_names=matrix.columns.values,
)
gx.test()
assignments = gx.assignments()

In [11]:
assignments

,cell_id,assignment,counts,pvalues,moi,n_umi,min_pvalue,max_count,log_odds,tested
0,AAACCCAAGACGCTCC-1,"[ENSA-4, EPB41L5-2]","[10, 501]","[1.3105544060377932e-05, 8.1585e-320]",2.0,511.0,8.158506e-320,501.0,723.485707,True
1,AAACCCAAGCTAATCC-1,[RELB-4],[515],[8.1585e-320],1.0,515.0,8.158506e-320,515.0,735.667566,True
2,AAACCCAAGGACTGGT-1,[CREB5-2],[9],[8.1585e-320],1.0,9.0,8.158506e-320,9.0,733.081359,True
3,AAACCCAAGGGAGGGT-1,[RPS20-2],[1072],[8.1585e-320],1.0,1072.0,8.158506e-320,1072.0,735.831453,True
4,AAACCCAAGTGCGTCC-1,[NR5A1-2],[155],[8.1585e-320],1.0,155.0,8.158506e-320,155.0,735.667566,True
...,...,...,...,...,...,...,...,...,...,...
25130,TTTGTTGTCAAGGACG-1,"[GLYR1-4, IFITM2-2, NR5A1-2]","[13, 11, 13]","[1.8681828261798048e-15, 1.0667612708672792e-1...",3.0,37.0,1.357556e-25,13.0,22.784792,True
25131,TTTGTTGTCAGGAACG-1,"[RELN-4, ZNF530-2]","[8, 42]","[2.5394439881415008e-17, 4.786580134288002e-83]",2.0,50.0,4.786580e-83,42.0,151.336745,True
25132,TTTGTTGTCCAAAGGG-1,[ENSA-4],[61],[8.1585e-320],1.0,61.0,8.158506e-320,61.0,735.504871,True
25133,TTTGTTGTCCCTCATG-1,[ZNF530-2],[141],[8.1585e-320],1.0,141.0,8.158506e-320,141.0,735.667566,True


In [15]:
assignmet_dict = dict(zip(assignments['cell_id'], assignments['assignment']))

In [22]:
# for each gene list in assignment column, there is a list of p-values in pvalues column (in numpy array), lets make a dictioanry of cell_id: {genes:p-values}
pval_dict = dict(zip(assignments['cell_id'], assignments['pvalues']))
cell_list, gene_list, pval_list = [], [], []
for cell in assignmet_dict:
    genes = assignmet_dict[cell]
    pvals = pval_dict[cell]
    # repeat the cell_id for the length of genes
    cell = [cell] * len(genes)
    # append to list
    cell_list.extend(cell)
    gene_list.extend(genes)
    pval_list.extend(pvals)

# make a dataframe
df = pd.DataFrame({'cell_id': cell_list, 'gene': gene_list, 'pval': pval_list})
df


,cell_id,gene,pval
0,AAACCCAAGACGCTCC-1,ENSA-4,1.310554e-05
1,AAACCCAAGACGCTCC-1,EPB41L5-2,8.158506e-320
2,AAACCCAAGCTAATCC-1,RELB-4,8.158506e-320
3,AAACCCAAGGACTGGT-1,CREB5-2,8.158506e-320
4,AAACCCAAGGGAGGGT-1,RPS20-2,8.158506e-320
...,...,...,...
33596,TTTGTTGTCAGGAACG-1,RELN-4,2.539444e-17
33597,TTTGTTGTCAGGAACG-1,ZNF530-2,4.786580e-83
33598,TTTGTTGTCCAAAGGG-1,ENSA-4,8.158506e-320
33599,TTTGTTGTCCCTCATG-1,ZNF530-2,8.158506e-320


In [29]:
# remove the -num from gene name
df['gene'] = df['gene'].str.split('-').str[0]
df

,cell_id,gene,pval
0,AAACCCAAGACGCTCC-1,ENSA,1.310554e-05
1,AAACCCAAGACGCTCC-1,EPB41L5,8.158506e-320
2,AAACCCAAGCTAATCC-1,RELB,8.158506e-320
3,AAACCCAAGGACTGGT-1,CREB5,8.158506e-320
4,AAACCCAAGGGAGGGT-1,RPS20,8.158506e-320
...,...,...,...
33596,TTTGTTGTCAGGAACG-1,RELN,2.539444e-17
33597,TTTGTTGTCAGGAACG-1,ZNF530,4.786580e-83
33598,TTTGTTGTCCAAAGGG-1,ENSA,8.158506e-320
33599,TTTGTTGTCCCTCATG-1,ZNF530,8.158506e-320


In [30]:
# lets make a dictionary of gene: [cell_ids]
gene_dict = {}
for gene in df['gene'].unique():
    gene_dict[gene] = df[df['gene'] == gene]['cell_id'].tolist()
gene_dict

{'ENSA': ['AAACCCAAGACGCTCC-1',
  'AAATGGAAGCCTTTGA-1',
  'AACAACCCAGAGAATT-1',
  'AACAAGATCTCGGGAC-1',
  'AACCAACGTACTCGCG-1',
  'AACCCAAAGCGCATCC-1',
  'AACGAAAGTACCCAGC-1',
  'AACGGGAGTTACTCAG-1',
  'AACGTCAGTCGAAGCA-1',
  'AAGACTCGTCCCACGA-1',
  'AAGATAGAGCATGATA-1',
  'AAGATAGAGTTAGTGA-1',
  'AAGCCATTCAGCACCG-1',
  'AAGGTAACAGTCGTTA-1',
  'AAGTACCGTGCAGTGA-1',
  'AATCGTGAGAAATCCA-1',
  'AATGAAGAGCACCGAA-1',
  'AATGACCAGACCACGA-1',
  'AATGACCAGTGCGCTC-1',
  'AATGACCCAGGTCAAG-1',
  'AATTCCTAGACGGTTG-1',
  'AATTCCTTCAGACCCG-1',
  'ACAACCATCTCGAGTA-1',
  'ACATCGAAGTCTGTAC-1',
  'ACCAACATCCTTGACC-1',
  'ACCTGTCGTTCCAAAC-1',
  'ACGATCACAGTAGTTC-1',
  'ACGATCATCCTGATAG-1',
  'ACGATGTCACCATATG-1',
  'ACGGAAGAGCCACCGT-1',
  'ACGGAAGGTCGAATTC-1',
  'ACGGTCGGTCCACAGC-1',
  'ACGGTTATCCCGTTCA-1',
  'ACTATGGGTGAACCGA-1',
  'ACTATGGGTGAACCGA-1',
  'ACTCCCAGTTAGCGGA-1',
  'ACTCTCGAGCCGGAAT-1',
  'ACTTCGCTCACCCTTG-1',
  'ACTTTGTTCCATTTAC-1',
  'AGAACAAGTAGACAAT-1',
  'AGAACCTAGCGACTTT-1',
  'AGAAC

In [32]:
# for each gene lets make a text file with cell_ids
for gene in gene_dict:
    with open(ROOT + '/gem0/perturbed_BCs/' + gene + '.txt', 'w') as f:
        for cell in gene_dict[gene]:
            f.write(cell + '\n')

In [34]:
# lets do everything now for gem1
gem_2 = pd.read_csv(ROOT + '/gem1/protospacer_calls_per_cell.csv')

gem_2['feature_call'] = gem_2['feature_call'].str.split('|')
gem_2['num_umis'] = gem_2['num_umis'].str.split('|')
gem_2 = gem_2.explode(list(['feature_call','num_umis']))
gem_2.drop(columns='num_features', inplace=True)
gem_2.rename(columns={"cell_barcode": "barcode", "feature_call": "guide", "num_umis": "n_umi"}, inplace=True)

# remove NA from matrix
gem_2 = gem_2[gem_2['guide'].notna()]
gem_2 = gem_2[gem_2['n_umi'].notna()]

# conver to cell by guide matrix, with n_umi as value, with NA as 0
gem_2_mat = gem_2.pivot(index='barcode', columns='guide', values='n_umi')
gem_2_mat.fillna(0, inplace=True)
# remove index and column name
gem_2_mat.index.name = None
gem_2_mat.columns.name = None


# set dtype of columns in gem_1, to dtype={"barcode": str, "guide": str, "n_umi": int},
gem_2 = gem_2.astype({"barcode": str, "guide": str, "n_umi": int})

matrix = gem_2.pivot_table(
    index="barcode", columns="guide", values="n_umi", fill_value=0
)

gx = Geomux(
    matrix,
    cell_names=matrix.index.values,
    guide_names=matrix.columns.values,
)
gx.test()
assignments = gx.assignments()

assignments

,cell_id,assignment,counts,pvalues,moi,n_umi,min_pvalue,max_count,log_odds,tested
0,AAACCCAAGATGACAT-1,"[PPP1R12A-4, PROX1-4]","[77, 389]","[4.916398630847478e-78, 8.4905e-320]",2.0,466.0,8.490518e-320,389.0,556.679219,True
1,AAACCCAAGATTAGTG-1,[KCNIP3-2],[477],[8.4905e-320],1.0,477.0,8.490518e-320,477.0,735.911576,True
2,AAACCCAAGCGCCTAC-1,[ZNF345-4],[717],[8.4905e-320],1.0,717.0,8.490518e-320,717.0,735.911576,True
3,AAACCCAAGCGTGTTT-1,[UBE2D2-4],[283],[8.4905e-320],1.0,283.0,8.490518e-320,283.0,735.911576,True
4,AAACCCAAGCTCGGCT-1,[KLF9-4],[16],[8.4905e-320],1.0,16.0,8.490518e-320,16.0,733.599453,True
...,...,...,...,...,...,...,...,...,...,...
25245,TTTGTTGTCAAACCCA-1,"[PHAX-4, ZNF345-2]","[107, 131]","[5.768635677323496e-185, 1.1722832678854901e-216]",2.0,238.0,1.172283e-216,131.0,72.973620,True
25246,TTTGTTGTCACTGGTA-1,[NFIB-4],[30],[8.4905e-320],1.0,30.0,8.490518e-320,30.0,734.416778,True
25247,TTTGTTGTCCCTCTAG-1,[ZNF530-4],[18],[8.4905e-320],1.0,18.0,8.490518e-320,18.0,733.744804,True
25248,TTTGTTGTCGCCTCTA-1,[HOXA3-4],[22],[8.4905e-320],1.0,22.0,8.490518e-320,22.0,734.003971,True


In [35]:
# continue from here
assignmet_dict = dict(zip(assignments['cell_id'], assignments['assignment']))

pval_dict = dict(zip(assignments['cell_id'], assignments['pvalues']))
cell_list, gene_list, pval_list = [], [], []
for cell in assignmet_dict:
    genes = assignmet_dict[cell]
    pvals = pval_dict[cell]
    cell = [cell] * len(genes)
    cell_list.extend(cell)
    gene_list.extend(genes)
    pval_list.extend(pvals)

df = pd.DataFrame({'cell_id': cell_list, 'gene': gene_list, 'pval': pval_list})

# remove the -num from gene name
df['gene'] = df['gene'].str.split('-').str[0]

gene_dict = {}
for gene in df['gene'].unique():
    gene_dict[gene] = df[df['gene'] == gene]['cell_id'].tolist()

for gene in gene_dict:
    with open(ROOT + '/gem1/perturbed_BCs/' + gene + '.txt', 'w') as f:
        for cell in gene_dict[gene]:
            f.write(cell + '\n')